# EncoderDecoder Sequence Fibrosis Progression

## 1. Libraries

In [1]:
#########################################################################
# 01. Libraries

import time
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import glob
from sklearn.model_selection import KFold, StratifiedKFold

import tensorflow as tf
# import tensorflow_addons as tfa
tf.keras.backend.clear_session()
import tensorflow_probability as tfp
tfd = tfp.distributions

# To allocate memory dynamically
physical_devices = tf.config.list_physical_devices('GPU')

try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    print('Invalid device or cannot modify virtual devices once initialized.')
# tf.config.experimental.enable_mlir_graph_optimization()

from tensorflow.keras import layers, models, optimizers, regularizers, constraints, initializers
from tensorflow.keras.utils import Sequence

from Utils.utils import *
from Utils.attention_layers import BahdanauAttention, ScaledDotProductAttention, GeneralAttention, VisualAttentionBlock
from Utils.preprocess_scans import *

pd.set_option('display.max_colwidth', 1000)

import warnings
warnings.filterwarnings("ignore")
import scipy as sp
import math
from functools import partial

#########################################################################

## 2. Global Variables

In [2]:
#########################################################################
# 02. Global Variables

path = '../01_Data/'
path_models = '../05_Saved_Models/'

path_train_masks = path + '/train_masks_fast_masks/'
path_test_masks = path + '/test_masks_fast_masks/'

path_scans_train = path + 'train/'
path_scans_test = path + 'test/'

#########################################################################

## 3. Load Data & Preprocess Data

In [3]:
##################################################################################################
# 03. Load Data & Preprocess Data

df_train = pd.read_csv( path + 'train.csv')
df_test = pd.read_csv(path + 'test.csv')

print(f'1.1 -> There are {df_train.Patient.unique().shape[0]} train unique patients')
print(f'1.2 -> There are {df_test.Patient.unique().shape[0]} test unique patients')

train_mask_paths = glob.glob(path_train_masks + '*')
test_mask_paths = glob.glob(path_test_masks + '*')

print(f'No. of Train Masks : {len(train_mask_paths)}')
print(f'No. of Test Masks : {len(test_mask_paths)}')
      
unique_train_patients = df_train.Patient.unique()
unique_test_patients = df_test.Patient.unique()

train_patients = os.listdir(path_train_masks)
test_patients = os.listdir(path_test_masks)

dict_train_patients_masks_paths = {patient: path_train_masks + patient + '/' for patient in train_patients}
dict_test_patients_masks_paths = {patient: path_test_masks + patient + '/' for patient in test_patients}

dict_train_patients_scans_paths = {patient: path_scans_train + patient + '/' for patient in unique_train_patients}
dict_test_patients_scans_paths = {patient: path_scans_test + patient + '/' for patient in unique_test_patients}

for patient in tqdm(dict_train_patients_masks_paths):
    list_files = os.listdir(dict_train_patients_masks_paths[patient])
    list_files = [dict_train_patients_masks_paths[patient] + file for file in list_files]
    dict_train_patients_masks_paths[patient] = list_files
    
for patient in tqdm(dict_test_patients_masks_paths):
    list_files = os.listdir(dict_test_patients_masks_paths[patient])
    list_files = [dict_test_patients_masks_paths[patient] + file for file in list_files]
    dict_test_patients_masks_paths[patient] = list_files
    

for patient in tqdm(dict_train_patients_scans_paths):
    list_files = os.listdir(dict_train_patients_scans_paths[patient])
    list_files = [dict_train_patients_scans_paths[patient] + file for file in list_files]
    dict_train_patients_scans_paths[patient] = list_files
    
for patient in tqdm(dict_test_patients_scans_paths):
    list_files = os.listdir(dict_test_patients_scans_paths[patient])
    list_files = [dict_test_patients_scans_paths[patient] + file for file in list_files]
    dict_test_patients_scans_paths[patient] = list_files
    
# Preprocessing:

df_train = df_train.groupby(['Patient', 'Weeks']).agg({
    'FVC': np.mean,
    'Percent': np.mean,
    'Age': np.max,
    'Sex': np.max,
    'SmokingStatus': np.max 
}).reset_index()

df_train['FVC_Percent'] = (df_train['FVC'] / df_train['Percent']) * 100
df_test['FVC_Percent'] = (df_test['FVC'] / df_test['Percent']) * 100

# Standarize data

mean_fvc, std_fvc = df_train.FVC.mean(), df_train.FVC.std()
mean_perc, std_perc = df_train.Percent.mean(), df_train.Percent.std()
mean_age, std_age = df_train.Age.mean(), df_train.Age.std()

df_train['Age'] = df_train['Age'].apply(lambda x: scale(x, mean_age, std_age))
df_test['Age'] = df_test['Age'].apply(lambda x: scale(x, mean_age, std_age))

df_train['FVC'] = df_train['FVC'].apply(lambda x: scale(x, mean_fvc, std_fvc))
df_test['FVC'] = df_test['FVC'].apply(lambda x: scale(x, mean_fvc, std_fvc))

df_train['FVC_Percent'] = df_train['FVC_Percent'].apply(lambda x: scale(x, mean_fvc, std_fvc))
df_test['FVC_Percent'] = df_test['FVC_Percent'].apply(lambda x: scale(x, mean_fvc, std_fvc))

df_train['Percent'] = df_train['Percent'].apply(lambda x: scale(x, mean_perc, std_perc))
df_test['Percent'] = df_test['Percent'].apply(lambda x: scale(x, mean_perc, std_perc))

# Mapping categories dictionaries 

dict_sex = {'Male': 0, 'Female': 1}
dict_sex_inv = {0: 'Male', 1: 'Female'}

dict_smoke = {'Ex-smoker': 0, 'Never smoked': 1, 'Currently smokes': 2}
dict_smoke_inv = {0: 'Ex-smoker', 1:'Never smoked', 2:'Currently smokes'}

dict_kind_patient = {'decreased': 0, 'regular': 1, 'increased': 2}
dict_kind_patient_inv = {0: 'decreased', 1: 'regular', 2: 'increased'}

df_train.Sex = df_train.Sex.apply(lambda x: dict_sex[x])
df_train.SmokingStatus = df_train.SmokingStatus.apply(lambda x: dict_smoke[x])

df_test.Sex = df_test.Sex.apply(lambda x: dict_sex[x])
df_test.SmokingStatus = df_test.SmokingStatus.apply(lambda x: dict_smoke[x])

# Build WeeksSinceLastVisit feature

df_train['ElapsedWeeks'] = df_train['Weeks']
df_test['ElapsedWeeks'] = df_test['Weeks']

train_weeks_elapsed = df_train.set_index(['Patient', 'Weeks'])['ElapsedWeeks'].diff().reset_index()
test_weeks_elapsed = df_test.set_index(['Patient', 'Weeks'])['ElapsedWeeks'].diff().reset_index()

df_train = df_train.drop('ElapsedWeeks', axis=1)
df_test = df_test.drop('ElapsedWeeks', axis=1)

train_weeks_elapsed['ElapsedWeeks'] = train_weeks_elapsed['ElapsedWeeks'].fillna(0).astype(int)
test_weeks_elapsed['ElapsedWeeks'] = test_weeks_elapsed['ElapsedWeeks'].fillna(0).astype(int)

df_train = df_train.merge(train_weeks_elapsed, how='inner', on=['Patient', 'Weeks'])
df_test = df_test.merge(test_weeks_elapsed, how='inner', on=['Patient', 'Weeks'])

df_train['patient_row'] = df_train.sort_values(['Patient', 'Weeks'], ascending=[True, True]) \
             .groupby(['Patient']) \
             .cumcount() + 1

df_test['patient_row'] = df_test.sort_values(['Patient', 'Weeks'], ascending=[True, True]) \
             .groupby(['Patient']) \
             .cumcount() + 1

df_train['WeeksSinceLastVisit'] = df_train.apply(lambda x: x['Weeks'] if x['patient_row']==1 else x['ElapsedWeeks'], axis=1)
df_test['WeeksSinceLastVisit'] = df_test.apply(lambda x: x['Weeks'] if x['patient_row']==1 else x['ElapsedWeeks'], axis=1)

# Norm Weeks

mean_weeks, std_weeks = df_train.Weeks.min(), df_train.Weeks.max()

df_train['WeeksSinceLastVisit'] = df_train['WeeksSinceLastVisit'].apply(lambda x: scale(x, mean_weeks, std_weeks))
df_test['WeeksSinceLastVisit'] = df_test['WeeksSinceLastVisit'].apply(lambda x: scale(x, mean_weeks, std_weeks))


df_train['Weeks'] = df_train['Weeks'].apply(lambda x: scale(x, mean_weeks, std_weeks))
df_test['Weeks'] = df_test['Weeks'].apply(lambda x: scale(x, mean_weeks, std_weeks))

# Ini dictionaries

columns = ['FVC', 'Age', 'Sex', 'SmokingStatus', 'WeeksSinceLastVisit', 'Percent']
dict_patients_train_ini_features, dict_patients_test_ini_features = {}, {}
dict_patients_train_kind_patient, dict_patients_test_kind_patient = {}, {}
df_train_patients, df_test_patients = df_train.set_index('Patient'), df_test.set_index('Patient')

for patient in unique_train_patients:
    dict_patients_train_ini_features[patient] = df_train_patients[columns][df_train_patients.index==patient].\
                                                                    to_dict('records')[0]
    std = np.std(unscale(df_train_patients['FVC'][df_train_patients.index==patient], mean_fvc, std_fvc).values)
    mean_first_1 = np.mean(unscale(df_train_patients['FVC'][df_train_patients.index==patient], mean_fvc, std_fvc).values[:1])
    mean_last_1 = np.mean(unscale(df_train_patients['FVC'][df_train_patients.index==patient], mean_fvc, std_fvc).values[-1:])
    if std<=100:
        dict_patients_train_kind_patient[patient] = 'regular'
    elif std>100 and mean_last_1 > mean_first_1 :
        dict_patients_train_kind_patient[patient] = 'increased'
    elif std>100 and mean_last_1 <= mean_first_1 :
        dict_patients_train_kind_patient[patient] = 'decreased'
    dict_patients_train_ini_features[patient]['kind'] = dict_kind_patient[dict_patients_train_kind_patient[patient]]
        
    
for patient in unique_test_patients:
    dict_patients_test_ini_features[patient] = df_test_patients[columns][df_test_patients.index==patient].\
                                                                    to_dict('records')[0]
    std = np.std(unscale(df_train_patients['FVC'][df_train_patients.index==patient], mean_fvc, std_fvc).values)
    mean_first_1 = np.mean(unscale(df_train_patients['FVC'][df_train_patients.index==patient], mean_fvc, std_fvc).values[:1])
    mean_last_1 = np.mean(unscale(df_train_patients['FVC'][df_train_patients.index==patient], mean_fvc, std_fvc).values[-1:])
    if std<=100:
        dict_patients_test_kind_patient[patient] = 'regular'
    elif std>100 and mean_last_1 > mean_first_1 :
        dict_patients_test_kind_patient[patient] = 'increased'
    elif std>100 and mean_last_1 <= mean_first_1 :
        dict_patients_test_kind_patient[patient] = 'decreased'
    dict_patients_test_ini_features[patient]['kind'] = dict_kind_patient[dict_patients_test_kind_patient[patient]]

# Decoder inputs

dict_train_sequence_fvc, dict_train_sequence_weekssincelastvisit = {}, {}
dict_train_sequence_cumweeks = {}
for patient in unique_train_patients:
    dict_train_sequence_fvc[patient] = list(df_train_patients['FVC'].loc[patient].values[1:])
    dict_train_sequence_weekssincelastvisit[patient] = list(df_train_patients['WeeksSinceLastVisit'].loc[patient].values[1:])
    dict_train_sequence_cumweeks[patient] = list(df_train_patients['Weeks'].loc[patient].values[1:])

##################################################################################################

100%|██████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 4890.75it/s]

1.1 -> There are 176 train unique patients
1.2 -> There are 5 test unique patients
No. of Train Masks : 176
No. of Test Masks : 5


## 4. Data Generator

Similar as `03_Autoencoder` Training Generator but instead of imgs as output we will have the ini features that we will use as our encoder input

In [4]:
#########################################################################

## 04. Data Generator

class ForecastTabularImgDataGenerator(Sequence):
    
    def __init__(self, raw_scans, training, patients, df_tabular, dict_ini_features, dict_patients_masks_paths,
                 batch_size=1, num_frames_batch=32, dict_raw_scans_paths=None, 
                 alpha=1.0, random_window=False, center_crop=True,
                 img_size_load=(500, 500, 3), 
                 img_size_crop=(440, 440, 3)):
        
        super(ForecastTabularImgDataGenerator, self).__init__()
        self.raw_scans = raw_scans
        self.training = training
        self.dict_ini_features = dict_ini_features
        self.batch_size = batch_size
        self.num_frames_batch = num_frames_batch
        self.alpha = alpha

        self.random_window = random_window
        self.center_crop = center_crop
        self.img_size_load = img_size_load
        self.img_size_crop = img_size_crop
        
        self.dict_patients_masks_paths = dict_patients_masks_paths
        self.dict_raw_scans_paths = dict_raw_scans_paths
        
        self.df_tabular = df_tabular
        self.ids = list(df_tabular.index)
        self.num_steps = int(np.ceil(len(self.ids) / self.batch_size))
        self.last_patient = ''
        self.on_epoch_end()
      
    # Number of batches in the sequence
    
    def __len__(self):
        return self.num_steps
    
    
    # Gets the batch at position index, return patient images and dict ini features
    
    def __getitem__(self, idx):
        indexes = self.indexes[idx*self.batch_size:(idx+1)*self.batch_size]
        patient_ids = list(self.df_tabular['Patient'].loc[indexes].unique()) # [self.ids[k] for k in indexes]
        if not self.raw_scans:
            list_scan_imgs = [decodePatientImages(patient, 
                                                  self.dict_patients_masks_paths,
                                                  image_size=(self.img_size_load[0], self.img_size_load[1]), 
                                                  numpy=True) 
                              for patient in patient_ids]
        else:
            list_scan_imgs = self.preprocessRawScans(patient_ids)
           
        patient_imgs = self.groupImages(list_scan_imgs)
        patient_imgs = self.loadImagesAugmented(patient_imgs)

        for patient_ in patient_ids:
            self.dict_ini_features[patient_]['Patient'] = patient_
        features = list(col for col in self.df_tabular.columns if col not in ['Patient', 'fvc_real'])

        patient_tabular_features = self.df_tabular[features].loc[indexes].values
        patient_y = np.expand_dims(self.df_tabular['fvc_real'].loc[indexes].values, -1)
        
        return (patient_imgs, patient_tabular_features, patient_y)
    
    
    # Preprocess Raw Scans in dicom format
    
    def preprocessRawScans(self, patient_ids):
        patients_files = [self.dict_raw_scans_paths[patient] for patient in patient_ids]
        patients_slices = [loadSlices(p) for p in patients_files]
        patients_images = [getPixelsHu(p_slices) for p_slices in patients_slices]
        patients_resampled_imgs = [resampleImages(p_images, p_slice, [1, 1, 1])[0] \
                                            for p_images, p_slice in zip(patients_images, patients_slices)]
        patients_crop_imgs = [np.asarray([imCropCenter(img, 320, 320) for img in p_resampled_imgs]) \
                              for p_resampled_imgs in patients_resampled_imgs]
        patients_segmented_lungs_fill = [np.asarray([seperateLungs(img, n_iters=2, only_internal=False, only_watershed=True)
                                                    for img in p_crop_imgs]) for p_crop_imgs in patients_crop_imgs]
        patients_masked_imgs = [np.where(p_lungs_fill==255, p_imgs, -2_048) \
                                for p_lungs_fill, p_imgs in zip(patients_segmented_lungs_fill, patients_crop_imgs)]
        
        patients_imgs = [windowImageNorm(p_imgs, min_bound=-1_000, max_bound=400) for p_imgs in patients_masked_imgs]
        patients_imgs = [tf.convert_to_tensor(img, dtype=tf.float32) for img in patients_imgs]
        patients_img_resized = [tf.convert_to_tensor([tf.image.resize(tf.expand_dims(img, axis=2), 
                                                                      (self.img_size_load[0], self.img_size_load[1])) 
                                                      for img in p_imgs], 
                                           dtype=tf.float32) for p_imgs in patients_imgs]
        return patients_img_resized
        
    
    # From n patient frames we will only keep self.alpha*n frames, cutting on top and bottom
    
    def filterSlices(self, array_imgs):
        num_patient_slices = array_imgs.shape[0]
        beta = int(self.alpha * num_patient_slices)
        if beta % 2 != 0:
            beta += 1
        if num_patient_slices > self.num_frames_batch:
            if beta > self.num_frames_batch and self.alpha < 1:
                remove = int((num_patient_slices - beta)/2)
                array_imgs = array_imgs[remove:, :, :, :]
                array_imgs = array_imgs[:-remove:, :, :]

        return array_imgs
    
    # Skip frames unniformally according to self.num_frames_batch value
    
    def frameSkipImages(self, patient_imgs):
        num_patient_slices = patient_imgs.shape[0]
        frame_skip = num_patient_slices // self.num_frames_batch
        skipped_patient_imgs = np.zeros((self.num_frames_batch, self.img_size_load[0], self.img_size_load[1], 1))
        for i in range(self.num_frames_batch):
            skipped_patient_imgs[i] = patient_imgs[i*frame_skip]    
        return skipped_patient_imgs
    
    # Select a random window of patient frames, in case its images has more frames than self.num_frame_batch 
    
    def randomWindow(self, patient_imgs):
        windowed_imgs = np.zeros((self.num_frames_batch, patient_imgs.shape[1], patient_imgs.shape[2], 1))
        num_frames = patient_imgs.shape[0]
        if num_frames < self.num_frames_batch:
            windowed_imgs[:num_frames] = patient_imgs
        else:
            random_frames = np.arange(num_frames)
            index = np.random.randint(0, num_frames - self.num_frames_batch)
            windowed_imgs[0:] = patient_imgs[index:index+self.num_frames_batch]
        return windowed_imgs
            
    
    # Convert raw frames to a fix size array -> (batch_size, num_frames_batch, img_size_crop[0], img_size_crop[1], 1)
    
    def groupImages(self, list_scan_imgs):
        grouped_imgs = []
        for patient_imgs in list_scan_imgs:
            if patient_imgs.shape[1] > self.num_frames_batch:
                patient_imgs = self.filterSlices(patient_imgs)
            if self.random_window:
                patient_imgs = self.randomWindow(patient_imgs)
            else:
                patient_imgs = self.frameSkipImages(patient_imgs)
            grouped_imgs.append(patient_imgs)
        return np.asarray(grouped_imgs)
        
    # Performs augmentation operations conserving the 3D property on the z axis
    
    def loadImagesAugmented(self, patient_imgs):

        if self.center_crop: #self.img_size_load != self.img_size_crop:
            # patient_imgs = self.center3Dcropping(patient_imgs)
            if patient_imgs.shape[2] > self.img_size_crop[0] and patient_imgs.shape[3] > self.img_size_crop[1]:
                patient_imgs = self.random3DCropping(patient_imgs)
        if self.training and np.random.random() > 0.5:
            patient_imgs = np.fliplr(patient_imgs)
        if self.training and np.random.random() > 0.5:
            patient_imgs = np.flipud(patient_imgs)
        if self.training and np.random.random() > 0.5:
            patient_imgs = patient_imgs[:, :, ::-1]
        if self.training and np.random.random() > 0.5:
            patient_imgs = patient_imgs[:, ::-1, :]
        if self.training:
            patient_rotated_imgs= []
            angle = np.random.randint(-15, 15)
            for batch in range(patient_imgs.shape[0]):
                batch_imgs_rotated = np.asarray([ndimage.rotate(patient_imgs[batch, i], angle, order=1,
                                                                reshape=False) for i in range(patient_imgs.shape[1])])
                patient_rotated_imgs.append(batch_imgs_rotated)
            patient_imgs = np.asarray(patient_rotated_imgs) 
        return patient_imgs
    
    # gull Center 3d Cropping 
    
    def fullcenter3DCropping(self, patient_imgs):
        cropped_imgs = []
        for batch in range(patient_imgs.shape[0]):
            imgs = np.asarray([cropLung(patient_imgs[batch, img].squeeze()) for img in range(patient_imgs.shape[1])])
            cropped_imgs.append(imgs)

        return np.expand_dims(np.asarray(cropped_imgs), axis=-1)
    
    #Random Cropping 3D - change x, y axis but not z
    
    def random3DCropping(self, patient_imgs):
        w, h = self.img_size_crop[0], self.img_size_crop[1]
        x = np.random.randint(0, patient_imgs.shape[2] - w)
        y = np.random.randint(0, patient_imgs.shape[2] - h)
        patient_crop_imgs = patient_imgs[:, :, y:y+h, x:x+w]
        return patient_crop_imgs
    
    # Center 3D Cropping
    
    def center3Dcropping(self, patient_imgs):
        w, h = patient_imgs.shape[2] - 20, patient_imgs.shape[3] - 20
        img_height, img_width = patient_imgs.shape[2], patient_imgs.shape[3]
        left, right = (img_width - w) / 2, (img_width + w) / 2
        top, bottom = (img_height - h) / 2, (img_height + h) / 2
        left, top = round(max(0, left)), round(max(0, top))
        right, bottom = round(min(img_width - 0, right)), round(min(img_height - 0, bottom))
        patient_crop_imgs = patient_imgs[:, :, top:bottom, left:right]
        return patient_crop_imgs
    
    # We shuffle the data at the end of each epoch
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.ids))
        np.random.shuffle(self.indexes)
     
    # Get only one patient, for debugging or prediction
        
    def getOnePatient(self, patient_id):
        if not self.raw_scans:
            list_scan_imgs = [decodePatientImages(patient_id, 
                                                  self.dict_patients_masks_paths,
                                                  image_size=(self.img_size_load[0], self.img_size_load[1]), 
                                                  numpy=True)]
        else:
            list_scan_imgs = self.preprocessRawScans([patient_id])
            
        patient_imgs = self.groupImages(list_scan_imgs)
        patient_imgs = self.loadImagesAugmented(patient_imgs)
        self.dict_ini_features[patient_id]['Patient'] = patient_id
        return (patient_imgs, [self.dict_ini_features[patient_id]])

    
def buildDataSet(list_patients, dict_ini_features, dict_seq_cumweeks, 
                 training=True, predictions=None):
    
    dict_to_tree = {
        'Patient' : [],
        'Weeks_Elapsed_since_firstVisit': [],
        'Base_Percent' : [],
        'Age' : [],
        'Sex' : [],
        'Base_Week' : [],
        'Base_FVC' : [],
        'SmokingStatus' : []
    }

    if training:
        dict_to_tree['fvc_real'] = []
    

    for patient in tqdm(list_patients, position=0):
        
        dict_to_tree['Weeks_Elapsed_since_firstVisit'].extend([dict_seq_cumweeks[patient][i] \
                                            for i in range(len(dict_seq_cumweeks[patient]))])
        
        for i in range(len(dict_seq_cumweeks[patient])):
            dict_to_tree['Patient'].extend([patient])

            dict_to_tree['Base_Percent'].extend([dict_ini_features[patient]['Percent']])

            dict_to_tree['Age'].extend([dict_ini_features[patient]['Age']])

            dict_to_tree['Sex'].extend([dict_ini_features[patient]['Sex']])

            dict_to_tree['Base_Week'].extend([dict_ini_features[patient]['WeeksSinceLastVisit']])

            dict_to_tree['Base_FVC'].extend([dict_ini_features[patient]['FVC']])

            dict_to_tree['SmokingStatus'].extend([dict_ini_features[patient]['SmokingStatus']])


        if training:
            dict_to_tree['fvc_real'].extend(dict_train_sequence_fvc[patient])

    df_tree = pd.DataFrame.from_dict(dict_to_tree, orient='columns')
    
    return df_tree

#########################################################################

## 5. Model

In [5]:
class BackBoneModel(models.Model):
    def __init__(self, img_dim=128, tabular_dim=64, features_dim=[32, 16], 
                 dropouts=[0.3, 0.2],
                 l2_reg=1e-4, batch_norm=False, max_norm=1,
                 path_img_model='./', **kwargs):
        
        super(BackBoneModel, self).__init__(**kwargs, name='BackBoneModel')
        
        self.features_dim = features_dim
        self.dropouts = dropouts
        self.l2_reg = l2_reg
        self.max_norm = max_norm
        self.dense_img = layers.Dense(img_dim,
                                      activation=None,
                                      kernel_constraint=constraints.MaxNorm(self.max_norm),
                                      bias_constraint=constraints.MaxNorm(self.max_norm),
                                      kernel_regularizer=regularizers.l2(self.l2_reg), 
                                      bias_regularizer=regularizers.l2(self.l2_reg))
        
        self.emb_sex = layers.Embedding(input_dim=2, output_dim=20, 
                                       embeddings_regularizer=regularizers.l2(1e-4))
                                    
        self.emb_smoker = layers.Embedding(input_dim=3, output_dim=20,
                                           embeddings_regularizer=regularizers.l2(1e-4))
        
        self.dense_tab = layers.Dense(features_dim[0],
                                                  activation=None,
                                                  kernel_regularizer=regularizers.l2(self.l2_reg), 
                                                  bias_regularizer=regularizers.l2(self.l2_reg))
        
        self.batch_norm = batch_norm
        self.img_bn = layers.BatchNormalization()
        self.tab_bn = layers.BatchNormalization()
        self.dropouts = dropouts
        
        if self.features_dim:
            self.fcc_denses, self.fcc_batch_norms = self.stackDense()
        if len(self.dropouts)>1:
            self.fcc_dropouts = self.stackDropout()
    
        self.img_model = models.load_model(path_img_model, compile=False)
        
        self.output_1 = layers.Dense(3, activation='linear', kernel_regularizer=regularizers.l2(self.l2_reg))
        self.output_2 = layers.Dense(3, activation='relu', kernel_regularizer=regularizers.l2(self.l2_reg))
        
        self.output_total = layers.Lambda(lambda x: x[0] + tf.cumsum(x[1]), name='quantile_preds')
        
        
    def call(self, inputs, training):
        img_inputs, tabular_inputs = inputs
        x = img_inputs
        
        x = self.img_model(x, training)
        x = self.dense_img(x)
        if self.batch_norm:
            x = self.img_bn(x, training)
        img_features = tf.nn.relu(x)
        
        #####
        patient_sex = self.emb_sex(tabular_inputs[:, 3])
        patient_smoke = self.emb_smoker(tabular_inputs[:, 6])
        
        tab_features = tf.concat([patient_sex,
                                   patient_smoke,
                                   tf.expand_dims(tabular_inputs[:, 0], 1),
                                   tf.expand_dims(tabular_inputs[:, 1], 1),
                                   tf.expand_dims(tabular_inputs[:, 2], 1),
                                   tf.expand_dims(tabular_inputs[:, 4], 1),
                                   tf.expand_dims(tabular_inputs[:, 5], 1)], 
                axis=-1) 
        
        tab_features = self.dense_tab(tab_features)
        if self.batch_norm:
            tab_features = self.tab_bn(tab_features, training)
        tab_features = tf.nn.relu(tab_features)

        ####
        x = tf.concat([img_features, tab_features], axis=-1)
        ####
        
        if self.features_dim:
            for i, (fcc_dense, fcc_drop) in enumerate(zip(self.fcc_denses, self.fcc_dropouts)):
                if len(self.dropouts)>1:
                    x = fcc_drop(x, training)
                x = fcc_dense(x)
                if self.batch_norm:
                    x = self.fcc_batch_norms[i](x, training)
                x = tf.nn.relu(x)
                
        ####
        output_1 = self.output_1(x)
#         output_2 = self.output_2(x)
#         output_total = self.output_total([output_1, output_2])
        ####
        
        return output_1
    
   
    def stackDropout(self):
        drops = []
        for rate in self.dropouts:
            d = layers.Dropout(rate)
            drops.append(d)
        return drops
        
        
    def stackDense(self):
        denses, batch_norms = [], []
        for units in self.features_dim:
            dense_ = layers.Dense(units,
                                   activation=None,
                                   kernel_constraint=constraints.MaxNorm(self.max_norm),
                                   bias_constraint=constraints.MaxNorm(self.max_norm),
                                   kernel_regularizer=regularizers.l2(self.l2_reg), 
                                   bias_regularizer=regularizers.l2(self.l2_reg))
            batch_norms.append(layers.BatchNormalization())
            denses.append(dense_)
        return denses, batch_norms
    
    

class PulmonarFibrosisClassicModel(models.Model):
    
    def __init__(self, img_dim, features_dim, tabular_dim, dropouts, l2_reg, max_norm, batch_norm, path_img_model,
                 learning_rate, clipvalue, quantiles, lambda_factor, beta_factor, 
                 first_epoch_learning_rate_epoch_decay, constant_learning_rate_epoch_decay, **kwargs):
        super(PulmonarFibrosisClassicModel, self).__init__(**kwargs, name='PulmonarFibrosisClassicModel')
        tf.keras.backend.clear_session()
        
        self.img_dim = img_dim
        self.tabular_dim = tabular_dim
        self.features_dim = features_dim
        self.dropouts = dropouts
        self.l2_reg = l2_reg
        self.max_norm = max_norm
        self.batch_norm = batch_norm
        self.path_img_model = path_img_model
        
        self.learning_rate = learning_rate
        self.first_epoch_learning_rate_epoch_decay = first_epoch_learning_rate_epoch_decay
        self.constant_learning_rate_epoch_decay = constant_learning_rate_epoch_decay 
        self.clipvalue = clipvalue
        self.quantiles = tf.constant(quantiles, dtype=tf.float32)
        self.lambda_factor = lambda_factor 
        self.beta_factor = beta_factor
        
        self.buildModel()
        self.compile()
        
        
    def compile(self):
        super(PulmonarFibrosisClassicModel, self).compile()
        
        self.optimizer = optimizers.Adam(learning_rate=self.learning_rate, 
                                         clipvalue=self.clipvalue)
#         self.optimizer=optimizers.Adam(lr=self.learning_rate, beta_1=0.9, beta_2=0.999, 
#                                       epsilon=None, decay=0.01, amsgrad=False, clipvalue=self.clipvalue)
        
        self.qloss = quantileLoss
        self.scoreloss = customLossFunction
        self.metric = [tf.keras.losses.MeanAbsoluteError(name='mae')]
        
#         self.compile(self.optimizer, self.qloss)
    
    def buildModel(self):
        self.backbone_model = BackBoneModel(img_dim=self.img_dim, tabular_dim=self.tabular_dim,
                                            features_dim=self.features_dim, 
                                            dropouts=self.dropouts,
                                            l2_reg=self.l2_reg, batch_norm=self.batch_norm, 
                                            max_norm=self.max_norm,
                                            path_img_model=self.path_img_model) 
    
    def learningRateDecay(self, epoch):
        if epoch == 0:
            self.optimizer.learning_rate = self.optimizer.learning_rate * self.first_epoch_learning_rate_epoch_decay
        else:
            self.optimizer.learning_rate = self.optimizer.learning_rate * self.constant_learning_rate_epoch_decay
            
    @tf.function    
    def train_step(self, backbone_inputs, target):
        
        with tf.GradientTape() as tape:
            y_pred = self.backbone_model(backbone_inputs, training=True)
            y_pred=tf.cast(y_pred, dtype=tf.float32)
            target=tf.cast(target, dtype=tf.float32)
            loss_1 = self.qloss(self.quantiles, 
                                unscale(target, mean_fvc, std_fvc), 
                                unscale(y_pred, mean_fvc, std_fvc))

            loss_2 = self.scoreloss(unscale(target, mean_fvc, std_fvc), 
                                    unscale(y_pred[:, 1], mean_fvc, std_fvc),  
                                    std=unscale(y_pred[:, 2], mean_fvc, std_fvc) -
                                        unscale(y_pred[:, 0], mean_fvc, std_fvc))

            loss = ((loss_1 * self.lambda_factor) + (loss_2 * (1-self.lambda_factor)))

            mae = self.metric[0](y_true=target, y_pred=y_pred[:, 1])

        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        
        return loss, loss_1, loss_2, mae

    
    def fitModel(self, X_train, X_val=None, epochs=1):
        history = {}
        history['loss'], history['val_loss'], history['metric'], history['val_metric'] = [], [], [], []
        history['val_Metrict3Timesteps'] = []
        
        total_loss, total_metric1, total_metric2, total_metric3 = 0, 0, 0, 0
        
        for epoch in range(epochs):
            start = time.time()
            print(f'Epoch [{epoch+1}/{epochs}]')
            len_X_val = 0 if X_val is None else len(X_val)
            len_X_train = len(X_train)
            pbar = tf.keras.utils.Progbar(len_X_train + len_X_val)
            
            for num_batch, batch in enumerate(X_train):
                img_features, tabular_features, target = batch
                backbone_inputs = (img_features, tabular_features)
                
                loss, loss_1, loss_2, mae = self.train_step(backbone_inputs, target)
                
                pbar.update(num_batch + 1, values=[('Loss', loss)] + \
                                                  [('qloss', loss_1)]  + \
                                                  [('Metric', loss_2)] + \
                                                  [('mae', mae)]) 
                
                total_loss += loss
                total_metric1 += loss_1
                total_metric2 += loss_2
                total_metric3 += mae
                
            history['loss'].append(total_loss)
            history['metric'].append(total_metric2)
            
            # Validation
            if X_val:
                val_total_loss, val_total_metric, val_total_metric2 = 0, 0, 0
                for num_batch, batch in enumerate(X_val):
                    img_features, tabular_features, val_target = batch
                    backbone_inputs = (img_features, tabular_features)
                    
                    y_val_pred = self.backbone_model(backbone_inputs, training=False)
                    
                    val_loss_1 = self.qloss(self.quantiles, 
                                        unscale(val_target, mean_fvc, std_fvc), 
                                        unscale(y_val_pred, mean_fvc, std_fvc))

                    val_loss_2 = self.scoreloss(unscale(val_target, mean_fvc, std_fvc),
                                             unscale(y_val_pred[:, 1], mean_fvc, std_fvc),  
                                             std=unscale(y_val_pred[:, 2], mean_fvc, std_fvc) -
                                                 unscale(y_val_pred[:, 0], mean_fvc, std_fvc))

                    val_loss = ((val_loss_1 * self.lambda_factor) + (val_loss_2 * (1-self.lambda_factor)))
                    
                    val_mae = self.metric[0](y_true=val_target, y_pred=y_val_pred[:, 1])
                    
                    pbar.update(len_X_train + num_batch + 1, values=[('val_Loss', val_loss)] + \
                                                                    [('val_qloss', val_loss_1)]  + \
                                                                    [('val_Metric', val_loss_2)] + \
                                                                    [('val_mae', val_mae)]) 

                    val_total_loss += val_loss
                    val_total_metric += val_loss_1
                    
                val_total_loss  /= float(len_X_val)
                val_total_metric /= float(len_X_val)
                history['val_loss'].append(val_total_loss)
                history['val_metric'].append(val_total_metric)
                
            self.learningRateDecay(epoch)
            X_train.on_epoch_end() 
            if X_val:
                X_val.on_epoch_end()
            print(' ({:.0f} sec)\n'.format( time.time() - start))
        return history
            
            
    @tf.function
    def test_step(self, data):
        pass
    
    

---

In [6]:
model_inputs = dict(
    img_dim=128, 
    tabular_dim=64,
    features_dim=[32, 16], 
    dropouts=[0.3, 0.2],
    l2_reg=1e-4, 
    max_norm=0.1,
    batch_norm=False,
    path_img_model=path_models + 'customModel',
    
    learning_rate=2e-3,
    first_epoch_learning_rate_epoch_decay=0.9,
    constant_learning_rate_epoch_decay=0.9,
    
    clipvalue=0.5,
    lambda_factor=0.8,
    beta_factor=0.6,
    quantiles=[0.2, 0.5, 0.8]
)

## 6. Model Training

In [7]:
img_size_load=(260, 260, 1)
img_size_crop=(220, 220, 1)
num_frames_batch = 32
train_alpha = 0.7
val_alpha = 0.7
batch_size = 1

skf = StratifiedKFold(n_splits = 7, random_state = 12, shuffle = True)
list_models, list_history = [], []

for num_fold, (train_index, val_index) in enumerate(skf.split(unique_train_patients, 
                                                              np.zeros(unique_train_patients.shape[0]))):

    x_train_patients = list(unique_train_patients[train_index])
    x_val_patients = list(unique_train_patients[val_index])
    
    print(f'Num Fold: {num_fold + 1}')
    print(f'Train patients: {len(x_train_patients)}, Test patients: {len(x_val_patients)}') 
    
    df_train_model = buildDataSet(x_train_patients,
                             dict_ini_features=dict_patients_train_ini_features, 
                             dict_seq_cumweeks=dict_train_sequence_cumweeks, 
                             training=True, 
                             predictions=None)
    
    df_val_model = buildDataSet(x_val_patients,
                             dict_ini_features=dict_patients_train_ini_features, 
                             dict_seq_cumweeks=dict_train_sequence_cumweeks, 
                             training=True, 
                             predictions=None)
    
    print(f'Train rows: {df_train_model.shape[0]}, Test rows: {df_val_model.shape[0]}')

    X_train_generator = ForecastTabularImgDataGenerator(raw_scans=False, training=True, 
                                                        patients=x_train_patients, df_tabular=df_train_model,
                                                        batch_size=batch_size, num_frames_batch=num_frames_batch, 
                                                        alpha=train_alpha, random_window=True, center_crop=True,
                                                        img_size_load=img_size_load, img_size_crop=img_size_crop,
                                                        dict_ini_features=dict_patients_train_ini_features, 
                                                        dict_patients_masks_paths=dict_train_patients_masks_paths,
                                                        dict_raw_scans_paths=None)

    X_val_generator = ForecastTabularImgDataGenerator(raw_scans=False, training=False, 
                                                      patients=x_val_patients, df_tabular=df_val_model,
                                                      batch_size=1, num_frames_batch=num_frames_batch, 
                                                      alpha=val_alpha, random_window=True, center_crop=True,
                                                      img_size_load=img_size_load, img_size_crop=img_size_crop,
                                                      dict_ini_features=dict_patients_train_ini_features, 
                                                      dict_patients_masks_paths=dict_train_patients_masks_paths,
                                                      dict_raw_scans_paths=None)


    model = PulmonarFibrosisClassicModel(**model_inputs)
    
    
    history = model.fitModel(
        X_train=X_train_generator,
        X_val=X_val_generator,
        epochs=8
    )
    
    list_models.append(model)
    list_history.append(history)
    

100%|██████████████████████████████████████████████████████████████████████████████████████████| 26/26 [00:00<?, ?it/s]

Num Fold: 1
Train patients: 150, Test patients: 26
Train rows: 1167, Test rows: 199


Epoch [1/8]

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

1366/1366 [==============================] - 762s 558ms/step - Loss: 172.4527 - qloss: 213.5752 - Metric: 7.9629 - mae: 0.6068 - val_Loss: 80.8077 - val_qloss: 99.1684 - val_Metric: 7.3648 - val_mae: 0.3024
 (762 sec)

Epoch [2/8]
1366/1366 [==============================] - 757s 554ms/step - Loss: 99.4285 - qloss: 122.4291 - Metric: 7.4260 - mae: 0.3586 - val_Loss: 99.5388 - val_qloss: 122.5698 - val_Metric: 7.4147 - val_mae: 0.3635
 (757 sec)

Epoch [3/8]
1366/1366 [=======================

100%|██████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:00<?, ?it/s]

 (757 sec)

Num Fold: 2
Train patients: 151, Test patients: 25
Train rows: 1174, Test rows: 192


...

Epoch [1/8]

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

1366/1366 [==============================] - 757s 554ms/step - Loss: 147.3031 - qloss: 182.1557 - Metric: 7.8930 - mae: 0.5480 - val_Loss: 100.1857 - val_qloss: 123.3964 - val_Metric: 7.3431 - val_mae: 0.3470
 (757 sec)

Epoch [2/8]
1366/1366 [==============================] - 755s 552ms/step - Loss: 99.5581 - qloss: 122.5873 - Metric: 7.4412 - mae: 0.3601 - val_Loss: 86.8084 - val_qloss: 106.6953 - val_Metric: 7.2608 - val_mae: 0.2979
 (755 sec)

Epoch [3/8]
1366/1366 [=====================

100%|██████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:00<?, ?it/s]

 (755 sec)

Num Fold: 7
Train patients: 151, Test patients: 25
Train rows: 1173, Test rows: 193


Epoch [1/8]

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

1366/1366 [==============================] - 757s 554ms/step - Loss: 153.4107 - qloss: 189.7612 - Metric: 8.0090 - mae: 0.5292 - val_Loss: 78.5446 - val_qloss: 96.3529 - val_Metric: 7.3113 - val_mae: 0.2797
 (757 sec)

Epoch [2/8]
1366/1366 [==============================] - 752s 551ms/step - Loss: 99.5392 - qloss: 122.5698 - Metric: 7.4166 - mae: 0.3564 - val_Loss: 74.9878 - val_qloss: 91.9199 - val_Metric: 7.2597 - val_mae: 0.2736
 (752 sec)

Epoch [3/8]
1366/1366 [========================

KeyboardInterrupt: 

In [ ]:
# dict_num_visits = {}
# for patient in dict_train_sequence_fvc:
#     num_visits = len(dict_train_sequence_fvc[patient])
#     if num_visits not in dict_num_visits:
#         dict_num_visits[num_visits] = []
#     else:
#         dict_num_visits[num_visits].append(np.mean(unscale(np.array(dict_train_sequence_fvc[patient]), mean_fvc, std_fvc)))
        
# print(dict_num_visits)
# print('==='*20)
# for i in dict_num_visits:
#     print(i)
#     print(np.mean(dict_num_visits[i]))
#     print('==='*20)

In [10]:
val_loss = np.mean([history['val_loss'][-1] for history in list_history])
val_metric = np.mean([(history['val_metric'][-1]) for history in list_history])
# val_metric_last3 = np.mean([(history['val_Metrict3Timesteps'][-1]) for history in list_history])

print(val_loss, val_metric)

71.00115 86.966705


In [ ]:
### History models
# 1. val_loss - 0.14948401 & val_metric = 7.581691 | quantiles=[0.2, 0.5, 0.8], eps=0, eps_decay=0
# 2. 1.7004111 7.6539536 | quantiles=[0.2, 0.5, 0.8], eps=0, eps_decay=0. lfactor=0.8 & resnet=True & dim=128
# 3. 1.9819709 7.499232 | quantiles=[0.2, 0.5, 0.8], eps=0, eps_decay=0. lfactor=0.75 & resnet=False & dim=256 & visuallatt
# 4. 1.6195476 7.4542327 | quantiles=[0.2, 0.5, 0.8], eps=0, eps_decay=0. lfactor=0.8 & resnet=custom & dim=128
# 5. (7.30) 1.5714737 7.306921 | quantiles=[0.2, 0.5, 0.8], eps=0, eps_decay=0. lfactor=0.8 & resnet=custom & dim=128 & lrdecay=0.9
# 6. (Best - 7.00) | 1.5109245 7.061371 quantiles=[0.2, 0.5, 0.8], eps=0, eps_decay=0. lfactor=0.8 & resnet=custom & dim=128 & inidecay=0.5 & lrdecay=0.9
# 7. 1.4737307 6.9873514 7.0969524 | quantiles=[0.2, 0.5, 0.8], eps=0, eps_decay=0. lfactor=0.8 & beta_factor=0.6 & resnet=custom & dim=128 & inidecay=0.9 & lrdecay=0.9
# 8. 1.4489578 6.906362 6.972941 | Add kind patient feature
# 9. 1.425578 6.822274 6.857718 | Add kind and remove dropouts

---

## 7. Evaluation & Interpretability

In [ ]:
#########################################################################

def plotAttention(images, list_weeks_elapsed, result, attention_plot, alpha=0.7, max_imgs=False):
        
        fig = plt.figure(figsize=(12, 12))
        if max_imgs:
            temp_image = np.max(images, axis=0)
        len_result = len(result)
        for i in range(len_result):
            if not max_imgs:
                temp_image = images[i]
            temp_att = np.resize(attention_plot[i], (8, 8))
            if len_result >= 2:
                ax = fig.add_subplot(len_result//2, len_result//2, i+1)
            else:
                ax = fig.add_subplot(1, 1, 1)
            ax.set_title(f'Weeks: {list_weeks_elapsed[i]} - Pred: {int(result[i])}')
            img = ax.imshow(temp_image, cmap=plt.cm.bone)
            ax.imshow(temp_att, cmap='gray', alpha=alpha, extent=img.get_extent())

        plt.tight_layout()
        plt.show()


        
X_generator = SequenceToSequenceDataGenerator(raw_scans=False, training=False, patients=x_val_patients,
                                                  batch_size=1, num_frames_batch=32, 
                                                  alpha=val_alpha, random_window=True, center_crop=True,
                                                  img_size_load=img_size_load, img_size_crop=img_size_crop,
                                                  dict_ini_features=dict_patients_train_ini_features, 
                                                  dict_patients_masks_paths=dict_train_patients_masks_paths,
                                                  dict_raw_scans_paths=None)


patient = np.random.choice(unique_train_patients)
print(f'Patient: {patient}')
# # # patient = 'ID00267637202270790561585'
batch = X_train_generator.getOnePatient(patient)
list_weeks_elapsed = list(dict_train_sequence_weekssincelastvisit[patient])
list_weeks_cum = list(dict_train_sequence_cumweeks[patient])
result, confidences, attention_plot = model.predictEvaluateModel(X_generator=X_generator,
                                                      batch=None,
                                                      patient=patient, 
                                                      list_weeks_elapsed=list_weeks_elapsed, 
                                                      list_weeks_since_firstvisit=list_weeks_cum,
                                                      initial_fvc=np.asarray([dict_patients_train_ini_features[patient]['FVC']]))
                                                      # initial_fvc=np.asarray([scale(500, mean_fvc, std_fvc)]))

patient_imgs = batch[0]
plotAttention(patient_imgs[0].squeeze(), list_weeks_elapsed, 
              unscale(result.numpy(), mean_fvc, std_fvc), attention_plot, alpha=0.8, max_imgs=True)
plotSequencePrediction(unscale(result.numpy(), mean_fvc, std_fvc),
                       unscale(np.array(dict_train_sequence_fvc[patient]), mean_fvc, std_fvc), 
                       list_weeks_elapsed)

#########################################################################

---

## 8. Baselines

- 1. **Mean FVC** from training to all predictions.
- 2. **Initial FVC per Patient** from each patient we take initial fvc and predict it over all timesteps.
- 3. **Last timestep** (Although this is a good baseline for general Sequence-to-Sequence purposes it is not for this case because we want to focus on the prognosis availability of the model).

### Baseline 1- Mean FVC

In [ ]:
#########################################################################
# Predict always the train FVC mean

test_patients = list(df_test['Patient'].unique())
dict_predictions = {
    'Patient': [],
    'target': [],
    'prediction': []
}

global_train_mean_fvc = df_train.FVC.mean()

for i, patient in enumerate(test_patients):
    subset = df_train[df_train['Patient']==patient]
    list_weeks_elapsed = dict_train_sequence_weekssincelastvisit[patient]
    list_fvc_sequence = np.array(dict_train_sequence_fvc[patient])
    
    list_pred = list_fvc_sequence.copy()
    list_pred[:] = global_train_mean_fvc
    
    dict_predictions['Patient'].append(patient)
    dict_predictions['target'].append(unscale(list_fvc_sequence, mean_fvc, std_fvc).astype(int))
    dict_predictions['prediction'].append(unscale(list_pred, mean_fvc, std_fvc).astype(int))

df_base = pd.DataFrame({'Target': dict_predictions['target'], 
                        'Pred': dict_predictions['prediction']}, 
                       index=[dict_predictions['Patient']])

df_base['mse'] = df_base.apply(lambda x: np.mean((abs(np.array(x['Target']) - x['Pred'])**2)) ,axis=1)
df_base['rmse'] = df_base.apply(lambda x: np.sqrt(np.mean(abs(np.array(x['Target']) - x['Pred']))) ,axis=1)
df_base['mape'] = df_base.apply(lambda x: 100.0 * np.mean(abs((np.array(x['Target']) - x['Pred'])/np.array(x['Target']))),axis=1)
df_base['metric'] = df_base.apply(lambda x: np.mean([customLossFunction(x['Target'][i], 
                                                                        x['Pred'][i]).numpy() 
                                                     for i in range(len(x['Pred']))]),
                                  axis=1)
print('==='*20)
print('Metrics: ')
print(f"MSE: {np.mean(df_base['mse'])} - RMSE: {np.mean(df_base['rmse'])} - MAPE: {np.mean(df_base['mape'])}")
print(f"CustomMetric: {np.mean(df_base['metric'])}")

df_base

#########################################################################

### Baseline 2 - Initial FVC per Patient

In [ ]:
#########################################################################
# Predict always last Timestep

test_patients = list(df_test['Patient'].unique())
dict_predictions = {
    'Patient': [],
    'target': [],
    'prediction': []
}

for patient in test_patients:
    subset = df_train[df_train['Patient']==patient]
    list_weeks_elapsed = dict_train_sequence_weekssincelastvisit[patient]
    list_fvc_sequence = np.array(dict_train_sequence_fvc[patient])
    
    predictions = np.empty(list_fvc_sequence.shape)
    predictions[0:] = dict_patients_train_ini_features[patient]['FVC']
    
    dict_predictions['Patient'].append(patient)
    dict_predictions['target'].append(unscale(list_fvc_sequence, mean_fvc, std_fvc).astype(int))
    dict_predictions['prediction'].append(unscale(predictions, mean_fvc, std_fvc).astype(int))
    

df_base = pd.DataFrame({'Target': dict_predictions['target'], 
                        'Pred': dict_predictions['prediction']}, 
                       index=[dict_predictions['Patient']])

df_base['mse'] = df_base.apply(lambda x: np.mean((abs(np.array(x['Target']) - x['Pred'])**2)) ,axis=1)
df_base['rmse'] = df_base.apply(lambda x: np.sqrt(np.mean(abs(np.array(x['Target']) - x['Pred']))) ,axis=1)
df_base['mape'] = df_base.apply(lambda x: 100.0 * np.mean(abs((np.array(x['Target']) - x['Pred'])/np.array(x['Target']))),axis=1)
df_base['metric'] = df_base.apply(lambda x: np.mean([customLossFunction(x['Target'][i], 
                                                                        x['Pred'][i]).numpy() 
                                                     for i in range(len(x['Pred']))]),
                                  axis=1)
print('==='*20)
print('Metrics: ')
print(f"MSE: {np.mean(df_base['mse'])} - RMSE: {np.mean(df_base['rmse'])} - MAPE: {np.mean(df_base['mape'])}")
print(f"CustomMetric: {np.mean(df_base['metric'])}")
df_base

#########################################################################

### Baseline 3 - Last Timestep

In [ ]:
#########################################################################
# Predict always initial FVC

test_patients = list(df_test['Patient'].unique())
dict_predictions = {
    'Patient': [],
    'target': [],
    'prediction': []
}

for patient in test_patients:
    subset = df_train[df_train['Patient']==patient]
    list_weeks_elapsed = dict_train_sequence_weekssincelastvisit[patient]
    list_fvc_sequence = np.array(dict_train_sequence_fvc[patient])
    
    list_pred = np.empty(list_fvc_sequence.shape)
    list_pred[0] = dict_patients_train_ini_features[patient]['FVC']
    list_pred[1:] = list_fvc_sequence[0:-1]
    
    dict_predictions['Patient'].append(patient)
    dict_predictions['target'].append(unscale(list_fvc_sequence, mean_fvc, std_fvc).astype(int))
    dict_predictions['prediction'].append(unscale(list_pred, mean_fvc, std_fvc).astype(int))
    

df_base = pd.DataFrame({'Target': dict_predictions['target'], 
                        'Pred': dict_predictions['prediction']}, 
                       index=[dict_predictions['Patient']])

df_base['mse'] = df_base.apply(lambda x: np.mean((abs(np.array(x['Target']) - x['Pred'])**2)) ,axis=1)
df_base['rmse'] = df_base.apply(lambda x: np.sqrt(np.mean(abs(np.array(x['Target']) - x['Pred']))) ,axis=1)
df_base['mape'] = df_base.apply(lambda x: 100.0 * np.mean(abs((np.array(x['Target']) - x['Pred'])/np.array(x['Target']))),axis=1)
df_base['metric'] = df_base.apply(lambda x: np.mean([customLossFunction(x['Target'][i], 
                                                                        x['Pred'][i]).numpy() 
                                                     for i in range(len(x['Pred']))]),
                                  axis=1)
print('==='*20)
print('Metrics: ')
print(f"MSE: {np.mean(df_base['mse'])} - RMSE: {np.mean(df_base['rmse'])} - MAPE: {np.mean(df_base['mape'])}")
print(f"CustomMetric: {np.mean(df_base['metric'])}")
df_base

#########################################################################

### Model Metrics

In [ ]:
#########################################################################
# Model Prediction

test_patients = list(df_test['Patient'].unique())
dict_predictions = {
    'Patient': [],
    'target': [],
    'prediction': [],
    'confidences' : []
}

for patient in test_patients:
    subset = df_train[df_train['Patient']==patient]
    list_weeks_elapsed = dict_train_sequence_weekssincelastvisit[patient]
    list_weeks_cum = list(dict_train_sequence_cumweeks[patient])
    list_fvc_sequence = np.array(dict_train_sequence_fvc[patient])
    
    result, stds, _ = model.predictEvaluateModel(X_generator=X_val_generator,
                                                    patient=patient, 
                                                    list_weeks_elapsed=list_weeks_elapsed,
                                                    list_weeks_since_firstvisit=list_weeks_cum,
                                                    initial_fvc=[dict_patients_train_ini_features[patient]['FVC']])
    
    predictions, confidences = np.empty(len(list_fvc_sequence)+1), np.empty(len(list_fvc_sequence)+1)
    targets = np.empty(len(list_fvc_sequence)+1)
    predictions[0], confidences[0] = dict_patients_test_ini_features[patient]['FVC'], 100.0
    targets[0] = dict_patients_test_ini_features[patient]['FVC']
    predictions[1:] = result.numpy().flatten()
    confidences[1:] = stds.numpy().flatten()#  * 100
    targets[1:] = list_fvc_sequence
    
    dict_predictions['Patient'].append(patient)
    dict_predictions['target'].append(unscale(targets, mean_fvc, std_fvc).astype(int))
    dict_predictions['prediction'].append(unscale(predictions, mean_fvc, std_fvc).astype(int))
    dict_predictions['confidences'].append(confidences)
    
    df_metrics = pd.DataFrame({'Target': dict_predictions['target'], 
                               'Pred': dict_predictions['prediction'],
                               'Confidences' :dict_predictions['confidences']}, 
                                index=[dict_predictions['Patient']])
    

df_metrics['mse'] = df_metrics.apply(lambda x: np.mean((x['Target'] - x['Pred'])**2), axis=1)
df_metrics['rmse'] = df_metrics.apply(lambda x: np.sqrt(np.mean(abs(x['Target'] - x['Pred']))) ,axis=1)
df_metrics['mape'] = df_metrics.apply(lambda x: 100.0 * np.mean(abs((x['Target'] - x['Pred'])/x['Target'])),axis=1)
df_metrics['metric'] = df_metrics.apply(lambda x: np.mean([customLossFunction(x['Target'][i], 
                                                                              x['Pred'][i],
                                                                              std=x['Confidences'][i]).numpy() 
                                                           for i in range(len(x['Pred']))]),
                                  axis=1)

print('==='*20)
print('Metrics: ')
print(f"MSE: {np.mean(df_metrics['mse'])} - RMSE: {np.mean(df_metrics['rmse'])} - MAPE: {np.mean(df_metrics['mape'])}")
print(f"CustomMetric: {np.mean(df_metrics['metric'])}")
df_metrics

#########################################################################

### CV - Model Metrics

In [ ]:
X_generator=test_patients = list(df_test['Patient'].unique())
dict_predictions = {
    'Patient': [],
    'target': [],
    'prediction': [],
    'confidences' : []
}

for patient in tqdm(test_patients, position=0):
    subset = df_train[df_train['Patient']==patient]
    list_weeks_elapsed = dict_train_sequence_weekssincelastvisit[patient]
    list_weeks_cum = list(dict_train_sequence_cumweeks[patient])
    list_fvc_sequence = np.array(dict_train_sequence_fvc[patient])
    
    list_results = [model.predictEvaluateModel(X_generator=X_val_generator,
                                              patient=patient, 
                                              list_weeks_elapsed=list_weeks_elapsed,
                                              list_weeks_since_firstvisit=list_weeks_cum,
                                              initial_fvc=[dict_patients_train_ini_features[patient]['FVC']])[0].numpy().flatten()
                    for model in list_models]
    array_results = np.asarray(list_results)
    result = array_results.mean(axis=0)
    stds = 100 - array_results.std(axis=0)

    predictions, confidences = np.empty(len(list_fvc_sequence)+1), np.empty(len(list_fvc_sequence)+1)
    targets = np.empty(len(list_fvc_sequence)+1)
    predictions[0], confidences[0] = dict_patients_test_ini_features[patient]['FVC'], 100.0
    targets[0] = dict_patients_test_ini_features[patient]['FVC']
    predictions[1:] = result
    confidences[1:] = stds
    targets[1:] = list_fvc_sequence
    
    dict_predictions['Patient'].append(patient)
    dict_predictions['target'].append(unscale(targets, mean_fvc, std_fvc).astype(int))
    dict_predictions['prediction'].append(unscale(predictions, mean_fvc, std_fvc).astype(int))
    dict_predictions['confidences'].append(confidences)
    
    df_metrics = pd.DataFrame({'Target': dict_predictions['target'], 
                               'Pred': dict_predictions['prediction'],
                               'Confidences' :dict_predictions['confidences']}, 
                                index=[dict_predictions['Patient']])
    

df_metrics['mse'] = df_metrics.apply(lambda x: np.mean((x['Target'] - x['Pred'])**2), axis=1)
df_metrics['rmse'] = df_metrics.apply(lambda x: np.sqrt(np.mean(abs(x['Target'] - x['Pred']))) ,axis=1)
df_metrics['mape'] = df_metrics.apply(lambda x: 100.0 * np.mean(abs((x['Target'] - x['Pred'])/x['Target'])),axis=1)
df_metrics['metric'] = df_metrics.apply(lambda x: np.mean([customLossFunction(x['Target'][i], 
                                                                              x['Pred'][i],
                                                                              std=x['Confidences'][i]).numpy() 
                                                           for i in range(len(x['Pred']))]),
                                  axis=1)

print('==='*20)
print('Metrics: ')
print(f"MSE: {np.mean(df_metrics['mse'])} - RMSE: {np.mean(df_metrics['rmse'])} - MAPE: {np.mean(df_metrics['mape'])}")
print(f"CustomMetric: {np.mean(df_metrics['metric'])}")
df_metrics

---

## 9. Conclusions  

- The Sequence to Sequence is able to forecast long sequences and dynamic, one of the biggest challenges on inference stage is the wide range of week a patient can attend the doctor.

- What can the model do that a baseline can not? 
    - Offer good forecasts for **long time fvc measurements** with accurate results, without depending of a fix windowing time visits.

    - **Interpret how the CT-Scan is affecting the forecasts.** Many linear/gradient models can exploit linear relationships between patients metadata and the elapsed time between visits with FVC measure, but our model is using CT-Scan features along with metadata to perform reliable and confident results that can answer questions and not only make forecasts. 
    
- Regularitzation and Data Augmentation is strongly important due to the lack of data we have in our dataset.